# Perturbation of Neoclassical Model with Deterministic Productivity shock

From Pablo Winant's course

Our goal here is to compute a $\textrm{\textbf{linear approximation of solution to the
neoclassical model, close ot the steady-state}}$.

- Definition:
$$ y_t = f(k_t) = \exp(z_t) k_t^\alpha \\
c_t = y_t - i_t $$

- Transition Equation > Deterministic Productivity shock
$$
k_t = (1-\delta) k_{t-1} + i_{t-1} \\
z_t = \rho z_{t-1}
$$

- Control $i_t\in[0, y_t = \exp(z_t)k_t^\alpha[$
  - or equivalently $c_t \in ]0, \exp(z_t) k_t^{\alpha}]$
  - 0 excluded because $U'(0) = \infty $ with $U(x) = \frac{x^{1-\gamma}}{1-\gamma}$

> $\textrm{\textbf{\color{blue}Objective function}}$ <br>

$$ 
V(k_0, z_0) :: \max_{ \color{red}{\{ c_t, k_{t+1} \}}_{t\geq 0} } \sum_{t} \beta^t U(c_t) \\ \; \\
s.c \; \begin{cases} \begin{align*} k_{t+1} & = (1-\delta) k_{t} + \exp(z_t) k_t^\alpha - c_t \\ z_t & = \rho z_{t-1} \end{align*} \\ c_t > 0, \; k_{t+1}\geq 0 \end{cases}
$$

soit

$$ 
V(k_0, z_0) :: \max_{ \color{red}{\{ c_t, i_t, k_{t+1} \}}_{t\geq 0} } \sum_{t} \beta^t U(c_t) \\ \; \\
s.c \; \begin{cases} \begin{align*} c_t & = \exp(z_t) k_t^\alpha - i_t \\ k_{t+1} & = (1-\delta) k_{t} + i_t \\ z_t & = \rho z_{t-1} \end{align*} \\ i_t \geq 0, \; k_{t+1}\geq 0 \end{cases}
$$

soit 

$$ 
V(k_0, z_0) :: \max_{ \color{red}{\{ i_t, k_{t+1} \}}_{t\geq 0} } \sum_{t} \beta^t U[ \exp(z_t) k_t^\alpha - i_t ] \\ \; \\
s.c \; \begin{cases} \begin{align*} k_{t+1} & = (1-\delta) k_{t} + i_t \\ z_t & = \rho z_{t-1} \end{align*} \\ i_t \geq 0, \; k_{t+1}\geq 0 \end{cases}
$$

- $\textrm{\textbf{Pre-determined / State variables}}$
    - $k_t$ and $z_t$ are pre-determined variables at date $t$ = **state variables**
    - $c_t$ and $i_t$ are optimized upon at date $t$ = **control variables**
    - Solve with Lagrangian or Bellman operator

> $\textrm{\color{blue}\textbf{Lagrangian solution}}$

$$ 
V(k_0, z_0) :: \max_{ \color{red}{\{ c_t, k_{t+1} \}}_{t\geq 0} } \sum_{t} \beta^t U(c_t) \\ \; \\

s.c \; \begin{cases} \begin{align*} 

k_{t+1} & = (1-\delta) k_{t} + \exp(z_t) k_t^\alpha - c_t & \Rightarrow & \; {\color{red}\lambda_t}\\
z_t & = \rho z_{t-1} \\

c_t & > 0 & \Rightarrow & \; {\color{red}\mu_t} \; (\geq 0)\\

c_t & \leq \exp(z_t) k_t^\alpha \rightarrow \exp(z_t) k_t^\alpha - c_t \geq 0 & \Rightarrow & \; {\color{red}\nu_t} \; (\geq 0)\\

k_{t+1} & \geq 0 & \Rightarrow & \; {\color{red}q_t} \; (\geq 0)\\
\end{align*} \end{cases}
$$

$$\Rightarrow \textrm{{6 variables}}$$ 

$$\forall t \geq 0 \qquad 
\mathcal{L}_t = \sum_{s \geq 0} \beta^{t+s}\left
\{ \; U(c_{t+s}) + {\color{green} \mu_s}  [c_{t+s}] + {\color{green} \nu_s} [\exp(z_{t+s})k_{t+s}^{\alpha} - c_{t+s}] + {\color{green} q_s}  [k_{t+s+1}] + {\color{green} \lambda_s} [(1-\delta) k_{t+s} + i_{t+s} - k_{t+s+1}] \; \right\}
$$

$$\textrm{\textbf{2 FOCs : }} \qquad \forall s \geq 0 \quad \begin{cases} \begin{align*} 
\frac{\partial \mathcal{L}_t}{\partial c_{t+s}} & = & 0 \\ 
\frac{\partial \mathcal{L}_t}{\partial k_{t+s+1}} & = & 0 \\
\end{align*} \end{cases}
$$

$$\textrm{\textbf{Karush-Kuhn-Tucker {\color{blue}complementary slackness condition} (at optima) : }}$$ 
$$
\begin{cases} \begin{align*}
{\color{blue}\mu_t} & = 0 \quad & \textrm{or} \quad & c_t = 0 & \rightarrow & \; \textrm{Impossible otherwise $U'(0) = \infty $ } \\
\nu_t & = 0 \quad & \textrm{or} \quad & c_t = \exp(z_t) k_t^\alpha & \rightarrow & \; \textrm{ ? } \\
{\color{blue}q_t} & = 0 \quad & \textrm{or} \quad & k_{t+1} = 0 & \rightarrow & \; \textrm{If $k_{t+1} = 0$ then so is $c_{t+1}$ which is impossible $U'(0) = \infty$ } \\
\end{align*} \end{cases}
$$
$$
\textrm{\textbf{\textit{and} }} \qquad (1-\delta) k_{t+s} + i_{t+s} - k_{t+s+1} = 0 \rightarrow {\color{blue}\lambda_t}
$$

- **Optimality Condition from FOCs** :
  - Takes into account the fact that $c_t>0$.
$$ \textrm{\textbf{Optimality condition - Euler equation : \quad }} 
\beta  \left[ \left( \frac{c_{t+1}}{c_t} \right) ^{-\gamma} \left( 1-\delta + \alpha e^{z_{t+1}} k_{t+1}^{\alpha -1} \right)\right] = 1
$$

> $\textrm{\textbf{\color{blue}Perturbation around the steady-state values}}$ 

- $z_{t}=\overline{z} + \Delta z_t \\$
- $k_{t}=\overline{k} + \Delta k_t \\$
- $i_{t}=\overline{i} + \Delta i_t \\$
- $c_{t}=\overline{c} + \Delta c_t$

Let 

$\overline{\Delta x_t} = \frac{\Delta x_t}{\overline{x}}$

- $\textrm{\textbf{\color{red}Linearize the model around the steady-state :}}$

$$\begin{align*}
" \beta  \left[ \left( \frac{c_{t+1}}{c_t} \right) ^{-\gamma} \left( 1-\delta + \alpha e^{z_{t+1}} k_{t+1}^{\alpha -1} \right)\right] & = 1 " \tag{1} \\ 
\rightarrow \beta  \left[ 
    \frac{(\overline{c}+ \Delta c_{t+1})^{-\gamma}}{( \overline{c} + \Delta c_t)^{-\gamma}} ( 1-\delta + \alpha e^{\overline{z} + \Delta z_{t+1}} (\overline{k} + \Delta k_{t+1})^{\alpha -1} )
    \right] 
    & = 1 \\ 
\beta  \left[
    (\overline{c}+ \Delta c_{t})^{\gamma} \frac{1}{(\overline{c} + \Delta c_{t+1})^{\gamma}} ( 1-\delta + \alpha e^{\overline{z} + \Delta z_{t+1}} (\overline{k} + \Delta k_{t+1})^{\alpha -1})
    \right] 
    & = 1 \nonumber \\ \\
\rightarrow \beta  \left[
    (1 + \gamma \overline{\Delta c_t}) (1 - \gamma \overline{\Delta c_{t+1}}) (1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}} (1+\Delta z_{t+1}) (1 + \overline{\Delta k_{t+1}})^{\alpha -1} )
    \right] 
    & \sim 1  \\
\beta  \left[
    1 + \gamma (\overline{\Delta c_t} - \overline{\Delta c_{t+1}}) \right] \left[1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}} (1+\Delta z_{t+1}) (1 + (\alpha -1) \overline{\Delta k_{t+1}})
    \right] 
    & \sim 1 \\
\beta  \left[
    1 + \gamma (\overline{\Delta c_t} - \overline{\Delta c_{t+1}}) \right] \left[1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}} (1+\Delta z_{t+1} + (\alpha -1) \overline{\Delta k_{t+1}})
    \right] 
    & \sim 1 \nonumber \\ \\
\left[
    1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}}
    \right] \left[ 1 + \frac{\gamma}{\overline{c}} (\Delta c_t - \Delta c_{t+1}) \right] + \alpha (\alpha-1) \overline{k}^{\alpha -1} e^{\overline{z}} \left[ \Delta z_{t+1} + \frac{\Delta k_{t+1}}{\overline{k}} \right]
    & \sim 1/\beta \\
\rightarrow \left[
    1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}}
    \right] \frac{\gamma}{\overline{c}} {\color{blue}(\Delta c_t - \Delta c_{t+1})} + \alpha (\alpha-1) \overline{k}^{\alpha -1} e^{\overline{z}} {\color{blue} \left[ \Delta z_{t+1} + \frac{\Delta k_{t+1}}{\overline{k}} \right]}
    & \sim 1/\beta - \left[
    1-\delta + \alpha \overline{k}^{\alpha -1} e^{\overline{z}}
    \right] \tag{1} \\
\end{align*}$$

$$ \begin{align*}
" c_t & = y_t - i_t "\tag{2} \\
\rightarrow \overline{c} + \Delta c_t & = e^{\overline{z}+ \Delta z_t} (\overline{k}+ \Delta k_t)^\alpha - \overline{i} - \Delta i_t \\
 & \sim e^{\overline{z}}(1+\Delta z_t) \overline{k}^{\alpha}(1 + \alpha\overline{\Delta k_t}) - \overline{i} - \Delta i_t \nonumber \\ \\
\rightarrow \overline{c} + {\color{blue}\Delta c_t} & \sim \overline{k}^{\alpha}e^{\overline{z}} \left[ {\color{blue}\Delta z_t + \frac{\Delta k_t}{\overline{k}}}\right] - \overline{i} - {\color{blue} \Delta i_t} \tag{2}
\end{align*}$$

- Transition (1)
$$\begin{align*}
"k_{t} & = (1-\delta) k_{t-1} + i_{t-1}" \tag{3} \\
\overline{k} + \Delta k_t & = (1-\delta) (\overline{k}+ \Delta k_{t-1}) + \overline{i }+ \Delta i_{t-1} \\
\rightarrow {\color{blue}\Delta k_t} & = (1-\delta) {\color{blue}\Delta k_{t-1} + \Delta i_{t-1}} -\delta \overline{k} + \overline{i} \tag{3} \\
\end{align*}$$

- Transition (2)
$$\begin{align*}
"z_{t} & = \rho z_{t-1}" \tag{4} \\
\overline{z} + \Delta z_t & = \rho (\overline{z} + \Delta z_{t-1}) \\
\rightarrow {\color{blue}\Delta z_t} & = \rho {\color{blue}\Delta z_{t-1}} + (\rho-1) \tag{4} \\
\end{align*}$$


> $\textrm{\textbf{\color{blue}General formulation of a \underline{Linearized model} / Formulation of the First Order System :}}$ 
>  - $s_t \in \mathbb{R}^{n_s}$ is a vector of ***states***
>  - $x_t \in \mathbb{R}^{n_x}$ is a vector of ***controls***
> $$ 
\begin{cases} \begin{align*} 
A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0_{n_x} \qquad \textrm{\textit{forward looking}} \\
s_{t+1} & = & E s_t + F x_t \qquad \textrm{\textit{backward looking}} 
\end{align*} \end{cases}
$$ 

$\textrm{\textbf{\underline{Linearization of the model around the Steady-State : }}}$ 

Let :
- $s_t = \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix}\quad$ : **state variables** > $k_t$ and $z_t$ are pre-determined variables at date $t$

- $x_t = \begin{bmatrix} \Delta i_t \\ \Delta c_t \end{bmatrix}\quad$ : **control variables** > $c_t$ and $i_t$ are optimized upon at date $t$

<br>

$\textrm{\textbf{The First Order System satisfies : }}$ 
$$ 
\textrm{\textbf{\underline{Around the steady-state :}}}\quad 
\begin{cases} \begin{align*} 
A \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix} + B \begin{bmatrix} \Delta i_t \\ \Delta c_t \end{bmatrix} + C \begin{bmatrix} \Delta z_{t+1} \\ \Delta k_{t+1} \end{bmatrix} + D \begin{bmatrix} \Delta i_{t+1} \\ \Delta c_{t+1} \end{bmatrix} 
& = 0 & \qquad \textrm{\textit{forward looking}} \\

\begin{bmatrix} \Delta z_{t+1} \\ \Delta k_{t+1} \end{bmatrix} & = E \begin{bmatrix} \Delta z_{t} \\ \Delta k_{t} \end{bmatrix} + F \begin{bmatrix} \Delta i_t \\ \Delta c_t \end{bmatrix} & \qquad \textrm{\textit{backward looking}} 
\end{align*} \end{cases}
$$

The linearized system is:
$$\begin{align*}
A & = & ...\\
B & = & ...\\
C & = & ...\\
D & = & ...\\
E & = & ...\\
F & = & ...
\end{align*}$$

$\textrm{\textbf{\underline{\color{blue}Solution of the First Order System around the Steady-State ? }}}$ 
- At date $t$ *controls* must be chosen as a function of (predetermined) *states*
- Mathematically speaking, the solution is a function $\varphi$ such that:
  $$\forall t, x_t = \varphi(s_t)$$
- **Since the model is linear** we look for un *unknown matrix* $X \in \mathbb{R}^{n_x} \times \mathbb{R}^{n_s}$ such that:
$${\color{red}x_t = X s_t}$$
- $\textrm{\textbf{\underline{In our Linearized model this means : }}}$ 
$$ {\color{red} \begin{bmatrix} \Delta i_t \\ \Delta c_t \end{bmatrix} = X \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix} } $$

----

In [1]:
using Symbolics, ForwardDiff, Random, Printf, Plots

**Create a NamedTuple to hold the model parameters.**

In [2]:
model = (; α=0.3, β=0.96, γ=4.0, δ=0.1, ρ=0.9)

(α = 0.3, β = 0.96, γ = 4.0, δ = 0.1, ρ = 0.9)

**Define two functions:** 
- `transition(z::Number, k::Number, i::Number, p)::Tuple{Number}` which
returns productivity and capital at date `t+1` as a function of
productivity, capital and investment at date `t` 
- `arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)::Number`
which returns the residual of the euler equation (lower case variable
for date t, upper case for date t+1)

$ \textrm{\underline{arbitrage} = Euler equation :} 
\begin{bmatrix} s_t \\ {\color{red}x_t} \\ s_{t+1} \\ {\color{red}x_{t+1}} \end{bmatrix}
= \begin{bmatrix} z_t \\ k_t \\ {\color{red}i_t} \\ z_{t+1} \\ k_{t+1} \\ {\color{red}i_{t+1}} \end{bmatrix} 
\mapsto \beta \left[ \frac{ e^{z_{t+1}} k_{t+1}^{\alpha} - i_{t+1} }{ e^{z_{t}} k_{t}^{\alpha} - i_{t} } \right]^{-\gamma} - 1 
$ 

**and**

$ \textrm{\underline{transition} :} 
\begin{bmatrix} s_t \\ {\color{red}x_t} \end{bmatrix}
= \begin{bmatrix} z_t \\ k_t \\ {\color{red}i_t} \end{bmatrix} 
\mapsto \begin{bmatrix} \rho z_t \\ (1-\delta)k_t + i_t \end{bmatrix}
= \begin{bmatrix} z_{t+1} \\ k_{t+1} \end{bmatrix}
= \begin{bmatrix} s_{t+1} \end{bmatrix}
$ 

In [3]:
function transition(z::Number, k::Number, i::Number, p)

    (; α, β, γ, δ, ρ) = p
    Z = ρ * z
    K = (1-δ)*k + i

    return (;Z,K)
end


function arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p)
    
    # α, β, δ, γ = p works but is order sensitive !! Below formulation reads : α corresponds to p.α whatever the order
    (; α, β, γ, δ, ρ) = p

    # Auxiliary variables today
    y = exp(z)*k^α
    c = y-i

    # Auxiliary variables tomorrow
    Y = exp(Z)*K^α
    C = Y-I

    # Residual of Euler equation
    residual = β*(C/c)^(-γ)*( 1-δ+α*exp(Z)*K^(α-1) ) - 1

    return residual
end

arbitrage (generic function with 1 method)

**Using multiple dispatch, define two variants of the same functions,
that take vectors as input and output arguments:** -
`arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number` -
`transition(s::Vector{T}, x::Vector{T}, p) where T<:Number`

In [4]:
#! /// To modify defined function to also accept different types than previously defined /// 

arbitrage(s::Vector{T}, x::Vector{T}, S::Vector{T}, X::Vector{T}, p) where T<:Number = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

# For transition, do not return the named tuple but a vector then
transition(s::Vector{T}, x::Vector{T}, p) where T<:Number = [ transition(s[1],s[2],x[1],p)... ]

# t = (3.0, 2.0, 1.0)
# [ x for x in t] to convert tuple into vector or simpler : [t...]

transition (generic function with 2 methods)

**Write a function `steady_state(p)::Tuple{Vector,Vector}` which
computes the steady-state of the model computed by hand.** It returns
two vectors, one for the states, one for the controls. Check that the
steady-state satisfies the model equations.

$\textrm{\textbf{\color{blue}\underline{Steady-state} }}$ 
- $\overline{c}, \overline{k}, \overline{z}, \overline{i}, \overline{y}$ such that:
  - $z_{t+1}=z_t=\overline{z}$
  - $k_{t+1}=k_t=\overline{k}$
  - $i_{t+1}=i_t=\overline{i}$
  - $c_{t+1}=c_t=\overline{c}$
  - ...
- ...satisfy the first order conditions i.e : 

$$\begin{cases} \begin{align*}
1 & = & \beta   \left( 1-\delta + \alpha e^{\overline{z}}{\overline{k}}^{\alpha -1} \right)  \\
\overline{k} & = & (1-\delta) \overline{k} + \overline{i} \\
\overline{z} & = & \rho \overline{z} \\
\overline{c} & = & e^{\overline{z}}\overline{k}^{\alpha} - \overline{i}
\end{align*}\end{cases} $$
- **Soit :**
$$\begin{cases} \begin{align*}
\overline{k} & = & \left( \frac{\frac{1}{\beta}-(1-\delta)}{\alpha} \right)^{\frac{1}{\alpha - 1}} \\
\overline{i} & = & \delta \overline{k} \\\
\overline{z} & = & 0 \\
\overline{c} & = & \overline{k}^{\alpha} - \overline{i}
\end{align*}\end{cases} $$

In [5]:
function steady_state(p)

    (; α, β, γ, δ, ρ) = p

    # Steady states
    z = 0.0
    k = ( (1/β - (1-δ))/α )^(1/(α-1))
    i = δ*k

    s = [z,k] # vector of states
    x = [i]   # vector of controls
    
    return (;
        s,
        x)
end

steady_state(model)

(s = [0.0, 2.920822149964071], x = [0.29208221499640713])

In [6]:
#? Check that the steady state is correct using the functions representing the model : i.e transition keeps still

ss = steady_state(model)
# transition(ss.s, ss.x, model) - ss.s 
# returns zero good

# Steady state values should satisfy Euler equation => residual = 0
arbitrage(ss.s, ss.x, ss.s, ss.x, model)
# returns zero good

# @assert maximum(transition(ss.s, ss.x, model) - ss.s) == 0.0

0.0

**Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.**

In [7]:
struct PerturbedModel
    A::Matrix
    B::Matrix
    C::Matrix
    D::Matrix
    E::Matrix
    F::Matrix
end

$\textrm{\textbf{{\color{blue}\underline{The first order system satisfies : linearized system around the steady state :}}}}$

We have $s_t = (\Delta z_t, \Delta k_t)$ and $x_t = (\Delta i_t)$

$$ 
\textrm{\textbf{\underline{Around the steady-state :}}}\quad 
\begin{cases} \begin{align*} 
A \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix} + B \begin{bmatrix} \Delta i_t \end{bmatrix} + C \begin{bmatrix} \Delta z_{t+1} \\ \Delta k_{t+1} \end{bmatrix} + D \begin{bmatrix} \Delta i_{t+1} \end{bmatrix} 
& = 0 & \qquad \textrm{\textit{forward looking}} \\

\begin{bmatrix} \Delta z_{t+1} \\ \Delta k_{t+1} \end{bmatrix} & = E \begin{bmatrix} \Delta z_{t} \\ \Delta k_{t} \end{bmatrix} + F \begin{bmatrix} \Delta i_t \end{bmatrix} & \qquad \textrm{\textit{backward looking}} 
\end{align*} \end{cases}
$$

<br>

$$ A = \begin{bmatrix} a_z & a_k \end{bmatrix} \quad B = [b_i] \quad C = \begin{bmatrix} c_z & c_k \end{bmatrix} \quad D = [d_i] \\ \; \\ E = \begin{bmatrix} e_{zz} & e_{zk} \\ e_{kz} & e_{kk} \end{bmatrix} \quad F = \begin{bmatrix} f_{zi} \\ f_{ki} \end{bmatrix} $$

<br>

**Write a function
`first_order_model(s::Vector, x::Vector, p)::PerturbedModel`, which
returns the *first order model*, given the steady-state and the
calibration. <br> Suggestion: use `ForwardDiff.jl` library.**

$ \textrm{\underline{arbitrage} = Euler :} 
\begin{bmatrix} s_t \\ {\color{red}x_t} \\ s_{t+1} \\ {\color{red}x_{t+1}} \end{bmatrix}
= \begin{bmatrix} z_t \\ k_t \\ {\color{red}i_t} \\ z_{t+1} \\ k_{t+1} \\ {\color{red}i_{t+1}} \end{bmatrix} 
\mapsto \beta \left[ \frac{ e^{z_{t+1}} k_{t+1}^{\alpha} - i_{t+1} }{ e^{z_{t}} k_{t}^{\alpha} - i_{t} } \right]^{-\gamma} - 1 
\quad \longleftrightarrow \quad
\textrm{\textit{\underline{Linearized / FO around the SS :}}} \; \; A s_t + B x_t + C s_{t+1} + D x_{t+1} = 0_{n_x=1} \quad \textrm{\textit{(forward looking)}}
$ 

**and**

$ \textrm{\underline{transition} :} 
\begin{bmatrix} s_t \\ {\color{red}x_t} \end{bmatrix}
= \begin{bmatrix} z_t \\ k_t \\ {\color{red}i_t} \end{bmatrix} 
\mapsto \begin{bmatrix} \rho z_t \\ (1-\delta)k_t + i_t \end{bmatrix}
= \begin{bmatrix} z_{t+1} \\ k_{t+1} \end{bmatrix}
= \begin{bmatrix} s_{t+1} \end{bmatrix}
\quad \longleftrightarrow \quad
\textrm{\textit{\underline{Linearized / FO around the SS :}}} \quad s_{t+1} = E s_t + F x_t \qquad \textrm{\textit{(backward looking)}} 
$ 

With : 

$$ A = \begin{bmatrix} a_z & a_k \end{bmatrix} \quad B = b_i \quad C = \begin{bmatrix} c_z & c_k \end{bmatrix} \quad D = d_i \\ \; \\ E = \begin{bmatrix} e_{zz} & e_{zk} \\ e_{kz} & e_{kk} \end{bmatrix} \quad F = \begin{bmatrix} f_{zi} \\ f_{ki} \end{bmatrix} $$

**Let** :

- $\mathcal{A_{rbi}} : (s_t,x_t,s_{t+1},x_{t+1}) \mapsto arbitrage(s_t,x_t,s_{t+1},x_{t+1}) = \beta \left[ \frac{ e^{z_{t+1}} k_{t+1}^{\alpha} - i_{t+1} }{ e^{z_{t}} k_{t}^{\alpha} - i_{t} } \right]^{-\gamma} - 1    \\ \; \\$
- $f : (s_t,x_t,s_{t+1},x_{t+1}) \mapsto A s_t + B x_t + C s_{t+1} + D x_{t+1} \qquad \Rightarrow \textrm{Linearization of } \mathcal{A_{rbi}} \textrm{ \textit{'arbitrage'} \textbf{\underline{around the SS}}} \\ \; \\$
- $\mathcal{T_{ransi}} : (s_t,x_t) \mapsto transition(s_t,x_t) = \begin{bmatrix} s_{t+1} \end{bmatrix} \\ \; \\$
- $g : (s_t,x_t) \mapsto E s_t + F x_t \qquad \Rightarrow \textrm{Linearization of } \mathcal{T_{ransi}} \textrm{ \textit{'transition'} \textbf{\underline{around the SS}}} \\$ 

<br>

**In particular we have, by evaluating the derivates AT the steady-state :**

$$\begin{cases} \begin{align*} 
\frac{\partial f}{\partial {\color{red}s_t} }& (s_t = \overline{s},\overline{x},\overline{s},\overline{x}) & = & {\color{red}A} & = & \frac{\partial \mathcal{A_{rbi}} }{\partial {\color{red}s_t} } (s_t = \overline{s},\overline{x},\overline{s},\overline{x})\\

\frac{\partial f}{\partial {\color{red}x_t} }& (\overline{s},x_t = \overline{x},\overline{s},\overline{x}) & = & {\color{red}B} & = & \frac{\partial \mathcal{A_{rbi}} }{\partial {\color{red}x_t} } (s_t = \overline{s},\overline{x},\overline{s},\overline{x})\\

\frac{\partial f}{\partial {\color{red}s_{t+1}} }& (\overline{s},\overline{x},s_{t+1} = \overline{s},\overline{x}) & = & {\color{red}C} & = & \frac{\partial \mathcal{A_{rbi}} }{\partial {\color{red}s_{t+1}} } (s_t = \overline{s},\overline{x},\overline{s},\overline{x})\\

\frac{\partial f}{\partial {\color{red}x_{t+1}} }& (\overline{s},\overline{x},\overline{s},x_{t+1} = \overline{x}) & = & {\color{red}D} & = & \frac{\partial \mathcal{A_{rbi}} }{\partial {\color{red}x_{t+1}} } (s_t = \overline{s},\overline{x},\overline{s},\overline{x})\\
\end{align*} \end{cases}

\qquad and \qquad

\begin{cases} \begin{align*} 
\frac{\partial g}{\partial {\color{red}s_t} }& (s_t = \overline{s},\overline{x}) & = & {\color{red}E} \\
\frac{\partial g}{\partial {\color{red}x_t} }& (\overline{s},x_t = \overline{x}) & = & {\color{red}F} \\
\end{align*} \end{cases}
$$

In [8]:
# We need to loosen the constraint on the arbitrage arguments : 

# brutal 
arbitrage(s, x, S, X, p) = [ arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p) ]
transition(s, x, p) = [ transition(s[1],s[2],x[1],p)... ]

# More precise, allows for each parameter to be a different type of Number
# arbitrage(s::Vector{<:Number}, x::Vector{<:Number}, S::Vector{<:Number}, X::Vector{<:Number}, p) = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

transition (generic function with 3 methods)

In [9]:
s = ss.s
x = ss.x

function first_order_model(s,x,parameters)

    A = ForwardDiff.jacobian( u->arbitrage(u,x,s,x,parameters), s)
    B = ForwardDiff.jacobian( u->arbitrage(s,u,s,x,parameters), x)
    C = ForwardDiff.jacobian( u->arbitrage(s,x,u,x,parameters), s)
    D = ForwardDiff.jacobian( u->arbitrage(s,x,s,u,parameters), x)
    E = ForwardDiff.jacobian( u->transition(u,x,parameters), s)
    F = ForwardDiff.jacobian( u->transition(s,u,parameters), x)

    return PerturbedModel(A,B,C,D,E,F)
end

FO_model = first_order_model(s,x,model)

PerturbedModel([5.074626865671642 0.5212190203776081], [-3.679193085018409;;], [-4.938626865671642 -0.5538125831185546], [3.679193085018409;;], [0.9 0.0; 0.0 0.9], [0.0; 1.0;;])

> **We look for a linear solution ${ \color{red} x_t = X s_t }$** <br> $\;$ <br>
> **In our case this amounts to** $\begin{bmatrix} \Delta c_t \\ \Delta i_t \end{bmatrix} = \begin{bmatrix} x_{cz} & x_{ck} \\ x_{iz} & x_{ik} \end{bmatrix} \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix}$ <br> $\;$ <br>
> **In thi code** $\begin{bmatrix} \Delta i_t \end{bmatrix} = {\color{red}\begin{bmatrix} x_z & x_k \end{bmatrix}} \begin{bmatrix} \Delta z_t \\ \Delta k_t \end{bmatrix}$ <br> $\;$ <br>
> $\textrm{\color{red} Equation satisfied by X ?}$ 

$$ 
\begin{cases} \begin{align*} 
A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0_{n_x} \qquad \textrm{\textit{forward looking}} \\
s_{t+1} & = & E s_t + F x_t \qquad \textrm{\textit{backward looking}} 
\end{align*} \end{cases}
$$

$s_{t+1} = E s_t + F x_t = (E + F.X) s_t$

$A s_t + B {\color{red}x_t} + C {\color{blue}s_{t+1}} + D {\color{green}x_{t+1}} 
= A s_t + B{\color{red}Xs_t} + C{\color{blue}(E+FX)s_t} + D{\color{green}Xs_{t+1}}
= A s_t + B{\color{red}Xs_t} + C{\color{blue}(E+FX)s_t} + D{\color{green}X(E+FX)s_t}
$

$\Rightarrow \forall s_t \quad \left[ A + BX + C(E+FX) + DX(E+FX) \right]s_t = 0 $

$$\Rightarrow \textrm{\textbf{\underline{X must satisfy :}}} \quad {\color{red} (A + B{\color{black}X}) + (C+D{\color{black}X})(E+F{\color{black}X}) = 0 } $$

**Write the matrix equation which `X` must satisfy** <br>
**Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`**

In [10]:
function residual(X::Matrix, M::PerturbedModel)
    # X must satisfy (A+BX) + (C+DX)(E+FX) = 0
    (; A,B,C,D,E,F) = M
    residual = A + B*X + (C+D*X)*(E+F*X)
    return residual
end

residual (generic function with 1 method)

$\textrm{\textbf{{\color{red}\underline{LINEAR TIME ITERATION ALGO}}}}$

Following *Pontus Rendahl (2017)*, our main equation of interest is : 

$\begin{cases}
A s_t + B x_t + C s_{t+1} + D x_{t+1} = 0 \\
s_{t+1} = E s_t + F x_t
\end{cases}$

**$\longrightarrow$ Suppose we have a guess $X^{{\color{red}(n)}}$ such that $\color{red} x_{t+1} = X^{(n)} s_{t+1}$ :** 

$\begin{cases}
A s_t + B x_t + C s_{t+1} + D X^{{\color{red}(n)}} s_{t+1} = 0 \\
s_{t+1} = E s_t + F x_t
\end{cases} \quad \Longrightarrow \quad
\begin{cases}
A s_t + B x_t + (C + D X^{{\color{red}(n)}}) (E s_t + F x_t) = 0 \\
s_{t+1} = E s_t + F x_t
\end{cases} \quad \Longrightarrow \quad
\begin{cases}
\left[ A + (C + D X^{{\color{red}(n)}})E \right] s_t + \left[ B + (C + D X^{{\color{red}(n)}})F \right] x_t = 0 \quad (*) \\
\longrightarrow \left[ B + (C + D X^{{\color{red}(n)}})F \right] x_t = - \left[ A + (C + D X^{{\color{red}(n)}})E \right] s_t \\
\longrightarrow  x_t = {\color{blue}- \left[ B + (C + D X^{(n)})F \right]^{-1}.\left[ A + (C + D X^{(n)})E \right]} s_t \\
\longrightarrow  x_t = {\color{blue}X^{(n+1)}} s_t \qquad \textrm{\color{blue}\underline{Updated guess}}\\
\end{cases}
$

**Write a function `T(X, M::PerturbedModel)` which implements the `time iteration step`.**

$ (*) \quad F(X,\tilde{X}) = ( A + (C + D X)E ) + ( B + (C + D \tilde{X})F ) X = 0\\$ 
$ \; \; \; \; \, \quad F(X,\tilde{X}) = (A+B X) + (C + D\tilde{X})(E + FX) = 0\\$ <br>
$\quad \Rightarrow X_{n+1}$ such that $F(X_{n+1}, X_{n}) = 0 \longrightarrow F(X_{n+1}:=T(X_n),X_n) = 0$



$$X_{n+1} := T(X_n) \longrightarrow F(X_{n+1},X_n) = 0$$

$$ F(X_{n+1}, X_n) = (A + B X_{n+1}) + ( C+ D X_n) ( E  + F X_{n+1} ) = 0 $$

$$\left[ B + (C+D X_{n})F \right] X_{n+1} + A  + (C+D X_n )E=0$$

$$ \longrightarrow X_{n+1} = \left[ B + (C + D X_n) F \right]^{-1} (A + (C+DX_n)E)$$

In [21]:
function T(X, M::PerturbedModel)
    (; A,B,C,D,E,F) = M
    T = - inv(B + (C+D*X)*F) * (A+(C+D*X)*E)
    return T
end
T(rand(1,2), FO_model)

1×2 Matrix{Float64}:
 2.25726  1.86966

**Write function
`linear_time_iteration(X_0::Matrix, M::PerturbedModel)::Matrix` which
implements the time iteration algorithm. <br>
Apply it to `X0 = rand(1,2)`
and check that the result satisfies the first order model.**

In [22]:
function linear_time_iteration(X_0::Matrix, M::PerturbedModel)
    ϵ = 10^(-16)    # erreur
    η = 10^(-14)    # tolerance convergence speed
    c = 1
    success_L = []
    cvspeed_L = []

    X = X_0
    Y = T(X,M)
    success = sum(residual(Y,M).^2)
    cvspeed = sum(abs.(Y - X))

    push!(cvspeed_L, cvspeed)
    push!(success_L, success)

    X = T(X_0,M)

    println("----------------------------------------------------------------------------------------------------------------------------")
    while (success>ϵ)&(cvspeed>η)
        if c>10000
            return "Iter > 10000"
        end
        c+=1
        Y = T(X,M)
        success = sum(residual(Y,M).^2)
        cvspeed = sum(abs.(Y - X))
        @printf "iter : %3.0f | Successive errors : %.14f | Success criterion : %9.6f \n" c cvspeed success
        push!(cvspeed_L, cvspeed)
        push!(success_L, success)
        X = copy(Y)
    end
    println("----------------------------------------------------------------------------------------------------------------------------")

    #fig = plot(legend=:topright, title="Linear Time Iteration algo", xaxis="Iterations", yaxis="F(X,X)")
    #plot!(fig, success_L, c=:blue)

    return Y
end

linear_time_iteration (generic function with 1 method)

In [20]:
linear_time_iteration(rand(1,2), FO_model)

----------------------------------------------------------------------------------------------------------------------------
iter :   2 | Successive errors : 8.76318519764042 | Success criterion : 72.155945 
iter :   3 | Successive errors : 1.34912419317484 | Success criterion :  3.076336 
iter :   4 | Successive errors : 0.41741163485160 | Success criterion :  0.697699 
iter :   5 | Successive errors : 0.22593322645118 | Success criterion :  0.287569 
iter :   6 | Successive errors : 0.15151920576413 | Success criterion :  0.156780 
iter :   7 | Successive errors : 0.11300205403421 | Success criterion :  0.098654 
iter :   8 | Successive errors : 0.08935154863220 | Success criterion :  0.067089 
iter :   9 | Successive errors : 0.07310636065883 | Success criterion :  0.047661 
iter :  10 | Successive errors : 0.06106536043929 | Success criterion :  0.034720 
iter :  11 | Successive errors : 0.05166312800231 | Success criterion :  0.025660 
iter :  12 | Successive errors : 0.0440549369

1×2 Matrix{Float64}:
 0.768674  0.0278097

**Define two linear operators
`L_S(U::Union{Vector, Matrix}, X_0::Matrix, m::PerturbedModel)::Matrix`
and `L_T(U::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which
implement the derivatives of the simulation and the time-iteration
operator respectively.**

**Implement a function `spectral_radius(f::Function)::Float64` which
implements the power iteration method to compute the biggest eigenvalues
of the two previously defined operators. Check that Blanchard-Kahn
conditions are met.**

**Write a function
`simulate(s0::Vector, X::Matrix, p, T::Int64)::Tuple{Matrix, Matrix}` to
simulate the model over $T$ periods (by using the formula
$\Delta s_t = (E + F X) \Delta s_{t-1}$. Return a matrix for the states
(one line per date) and another matrix for the controls. Bonus: add a
keyword option to compute variables levels or log-deviations. If
possible, return a DataFrame object.**

**Make some nice plots.**

------

**Warm-up: install the `ForwardDiff` library. Use it to differentiate
the function below. Check the jacobian function.**

Note: the signature of function `f` needs to be fixed first to allow for
dual numbers as arguments.

In [ ]:
test = eps(Float64)
println((1+test)-1)
println((1+test/2)-1)

# test = smallest number : a number smaller than test will disappear in computations

In [ ]:
@variables a b x

# Dual number :
# Pour redéfinir opérations 
# import Base: *
# import Base: /
# import Base: +
# import Base: -

a + b*x
Symbolics.derivative(sin(x^2), x)

In [ ]:
ForwardDiff.Dual(1.0,1.0)

In [ ]:
#function f(x::Vector{T}) where T <: Number # Is equivalent to below definition
function f(x::Vector{<:Number})
    #works because 'Dual' (number) of ForwardDiff is a Number !
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

In [ ]:
# typeof(ForwardDiff.Dual(2.0,1.0,0.0))
ForwardDiff.jacobian( u->f(u),[0.5,1.0] )